<a href="https://colab.research.google.com/github/anuraagshankar/Pasckathon_Bugslayers/blob/master/linear_fill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, Normalizer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, precision_recall_curve, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.utils import class_weight
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import gc
import pdb
from xgboost import XGBRegressor
import os
from tqdm.notebook import tqdm
import pickle
import pdb

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!unzip /content/drive/My\ Drive/sepsis/training_setA.zip 
!unzip /content/drive/My\ Drive/sepsis/training_setB.zip

In [ ]:
DIR1 = '/content/training/'
DIR2 = '/content/training_setB/'
files1 = os.listdir(DIR1)
files2 = os.listdir(DIR2)


In [ ]:
df = pd.read_csv(DIR1+files1[0], sep='|')
df

In [ ]:
df_linear_fill = df.interpolate().interpolate(limit_direction = 'backward')
df_linear_fill

In [ ]:
stats_df = pd.read_csv('/content/drive/My Drive/sepsis/stats.csv')

In [ ]:
cnt = 1
for f in tqdm(files1):
  df = pd.read_csv(DIR1+f, sep='|')
  df_linear_f = df.interpolate().interpolate(limit_direction = 'backward')
  na_cols = df_linear_f.columns[df_linear_f.isna().any()].tolist()
  for c in na_cols:
    val = float(stats_df[stats_df['column']==c]['mean'])
    df_linear_f[c].fillna(val, inplace=True)
  df_linear_f.to_csv(f'/content/drive/My Drive/sepsis/Linear-Fill/Linear-front-fill/{cnt}.csv',index=False)
  cnt += 1
for f in tqdm(files2):
  df = pd.read_csv(DIR2+f, sep='|')
  df_linear_f = df.interpolate().interpolate(limit_direction = 'backward')
  na_cols = df_linear_f.columns[df_linear_f.isna().any()].tolist()
  for c in na_cols:
    val = float(stats_df[stats_df['column']==c]['mean'])
    df_linear_f[c].fillna(val, inplace=True)
  df_linear_f.to_csv(f'/content/drive/My Drive/sepsis/Linear-Fill/Linear-front-fill/{cnt}.csv',index=False)
  cnt += 1

In [ ]:
FDIR = '/content/drive/My Drive/sepsis/Linear-Fill/Linear-front-fill/'
df_linear_f = pd.concat([pd.read_csv(FDIR+f) for f in tqdm(os.listdir(FDIR))])
df_linear_f.to_csv(FDIR+'dataset.csv',index=False)


# **Baseline**

In [ ]:
df_linear_f = pd.read_csv('/content/drive/My Drive/sepsis/Linear-Fill/linear_dataset.csv')

In [ ]:
df_linear_f.columns

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'HCO3', 'FiO2',
       'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium',
       'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate',
       'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI',
       'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'ICULOS',
       'SepsisLabel'],
      dtype='object')

In [ ]:
df_linear_f = df_linear_f.drop(columns=['Gender', 'Unit1', 'Unit2', 'BaseExcess', 'HospAdmTime', 'EtCO2'])

In [ ]:
cols = list(df_linear_f.columns)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df_linear_f)
base_df = scaler.transform(df_linear_f)
base_df = pd.DataFrame(base_df, columns = cols)
print(base_df.shape)

(1552210, 35)


In [ ]:
X = base_df.drop('SepsisLabel', axis=1)
y = base_df['SepsisLabel']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=50, class_weight='balanced')
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
print(roc_auc_score(y_test, y_pred))

0.88497077249355


**<h1>Correlation</h1>**

In [ ]:
df_corr = df_linear_f.copy()

In [ ]:
corr_mat_df = df_corr.corr()

In [ ]:
corr_mat_df.to_csv(FDIR+'correlation_matrix/Linear_fill_corr_mat.csv')

In [ ]:
corr_mat_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
HR,1.000000,-0.070411,0.240366,-0.037875,0.062572,0.108632,0.220770,0.001683,-0.010488,-0.031112,0.001298,-0.014311,0.001285,0.007016,0.013428,-0.023846,0.024253,-0.080220,0.001395,-0.021733,0.005059,0.069247,0.003620,-0.055740,-0.036697,0.009697,0.003964,0.010738,-0.104782,-0.106379,0.001192,0.125300,0.051115,0.038971,-0.156585,-0.028887,0.004294,-0.004294,-0.056177,0.047462,0.050702
O2Sat,-0.070411,1.000000,0.005688,0.040346,0.035256,0.009793,-0.137547,-0.003583,0.018124,-0.035405,-0.000888,0.067841,-0.081023,0.052658,-0.030435,-0.038283,-0.011171,-0.028704,0.093069,-0.005392,-0.006218,-0.021658,-0.095207,-0.021182,-0.042400,-0.024809,-0.005639,0.004136,-0.112304,-0.092017,-0.036852,-0.032643,-0.008428,-0.007369,-0.054425,-0.030028,-0.037297,0.037297,-0.008000,-0.042172,-0.008867
Temp,0.240366,0.005688,1.000000,0.030587,-0.032098,-0.064321,0.085878,0.021828,0.074584,0.022291,-0.003603,0.129380,-0.052462,0.032396,-0.000493,-0.158547,-0.045078,-0.066858,0.050110,-0.102688,-0.017600,-0.023625,-0.122270,-0.059862,-0.136159,-0.037118,-0.023378,0.020626,-0.096960,-0.069133,-0.087118,0.077540,0.047300,-0.026614,-0.093319,0.034662,-0.149715,0.149715,0.007048,0.090292,0.055568
SBP,-0.037875,0.040346,0.030587,1.000000,0.760823,0.485437,0.037593,0.010339,0.035179,0.018856,-0.002748,0.062735,-0.009838,0.065364,-0.018636,-0.004207,-0.023637,0.065983,-0.001446,0.073411,-0.020267,0.043490,-0.046474,-0.021930,-0.023490,-0.087258,-0.041120,-0.056846,0.079417,0.068693,-0.057657,-0.037176,0.008955,0.033234,0.026601,-0.012906,0.041712,-0.041712,0.037111,0.038338,-0.012483
MAP,0.062572,0.035256,-0.032098,0.760823,1.000000,0.739769,0.044258,0.011229,0.014801,-0.003131,-0.002374,0.048979,-0.010686,0.053244,-0.025657,-0.049536,-0.020466,0.076772,0.009857,0.068899,-0.027682,0.021924,-0.010051,-0.035113,-0.025074,-0.104324,-0.052066,-0.047169,0.172110,0.148641,-0.021734,-0.060175,0.010708,0.053586,-0.144574,0.002704,0.090730,-0.090730,0.041646,0.020834,-0.019493
DBP,0.108632,0.009793,-0.064321,0.485437,0.739769,1.000000,0.048265,0.010785,0.010563,0.017201,-0.001308,0.020080,0.004868,0.016815,-0.016667,-0.044133,-0.006073,0.094262,-0.011310,0.050249,-0.021092,0.001529,0.019753,-0.033758,-0.021822,-0.093386,-0.037444,-0.024055,0.214231,0.193325,-0.004056,-0.058999,0.007264,0.083731,-0.236528,0.024329,0.161468,-0.161468,0.043047,0.005530,-0.015420
Resp,0.220770,-0.137547,0.085878,0.037593,0.044258,0.048265,1.000000,-0.012587,0.013209,-0.000129,0.000291,0.021055,0.002321,-0.033447,-0.004023,0.054360,0.029152,0.033486,-0.042616,0.004799,0.009268,0.025893,0.017591,-0.000114,-0.021322,-0.002733,-0.007972,-0.004763,0.007336,-0.009199,0.011344,0.044797,0.038209,0.061996,0.034245,0.002066,0.077839,-0.077839,-0.024019,0.092780,0.041607
EtCO2,0.001683,-0.003583,0.021828,0.010339,0.011229,0.010785,-0.012587,1.000000,0.006288,0.004055,-0.001161,-0.009589,0.057045,0.005104,-0.027019,-0.007335,-0.013584,0.033522,-0.011815,-0.014391,-0.009250,0.001179,-0.040707,0.015823,-0.004500,-0.011661,-0.014010,-0.021508,0.021569,0.013945,-0.015588,-0.006932,0.011474,0.011361,-0.019781,-0.002152,0.003497,-0.003497,0.014684,0.061378,-0.006371
BaseExcess,-0.010488,0.018124,0.074584,0.035179,0.014801,0.010563,0.013209,0.006288,1.000000,0.663597,-0.001661,0.453187,0.341829,-0.001787,-0.060090,-0.084233,-0.019458,0.084180,-0.286794,-0.094835,-0.007925,-0.038654,-0.192873,-0.015016,-0.103957,-0.114858,-0.024636,0.006585,-0.006026,-0.000035,-0.065838,-0.055653,0.022645,0.071473,0.000562,0.019706,0.023995,-0.023995,-0.014731,0.101871,0.013043
HCO3,-0.031112,-0.035405,0.022291,0.018856,-0.003131,0.017201,-0.000129,0.004055,0.663597,1.000000,-0.000649,0.175639,0.400450,-0.004280,-0.047141,-0.